# Simple MNIST convnet

**Author:** [fchollet](https://twitter.com/fchollet)<br>
**Date created:** 2015/06/19<br>
**Last modified:** 2020/04/21<br>
**Description:** A simple convnet that achieves ~99% test accuracy on MNIST.

## Setup

In [16]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf

import numpy as np
import pandas as pd
import random
import pickle
import sklearn
from sklearn.metrics import accuracy_score
import os



## Prepare the data

In [5]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

x_train=x_train[1:6000,:,:]
y_train=y_train[1:6000]

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (5999, 28, 28, 1)
5999 train samples
10000 test samples


## Build the model

In [ ]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu", name='conv2'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

In [50]:
model = keras.Sequential(name="sequential1")
   
layer1=model.add(keras.Input(shape=input_shape))
layer2=model.add(layers.Conv2D(32, kernel_size=(3, 3), activation="relu"))
layer3=model.add(layers.MaxPooling2D(pool_size=(2, 2))
layer4=model.add(layers.Conv2D(64, kernel_size=(3, 3), activation="relu"))

layer5=model.add(layers.MaxPooling2D(pool_size=(2, 2)))
layer6=model.add(layers.Flatten())
layer7=model.add(layers.Dropout(0.5))
layer8=model.add(layers.Dense(num_classes, activation="softmax"))

model.summary()

SyntaxError: ignored

In [43]:
print(model.layers)

[<keras.layers.convolutional.Conv2D object at 0x7f23d766da90>, <keras.layers.pooling.MaxPooling2D object at 0x7f23d8907c10>, <keras.layers.convolutional.Conv2D object at 0x7f23d8907310>, <keras.layers.pooling.MaxPooling2D object at 0x7f23d76b8d10>, <keras.layers.core.flatten.Flatten object at 0x7f23d89371d0>, <keras.layers.core.dropout.Dropout object at 0x7f23d8937d10>, <keras.layers.core.dense.Dense object at 0x7f23d760e290>]


## Train the model

In [ ]:
batch_size = 128
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

## Evaluate the trained model

In [8]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.08362270146608353
Test accuracy: 0.974399983882904


In [9]:
model.save('MNIST6000_12_7_2021.model')

INFO:tensorflow:Assets written to: MNIST6000_12_7_2021.model/assets


In [13]:
new_model=tf.keras.models.load_model('MNIST6000_12_7_2021.model')

In [14]:
x_test=x_train[1:60,:,:]
y_train=y_train[1:60]

In [30]:
heatmap_df = pd.DataFrame()
signal_df = pd.DataFrame()
#for ih in range(1,1000):
for ih in range(1,10):
  
    predictions=new_model.predict(x_test)
#    print(predictions)
    #------------------------------------------------------
    with tf.GradientTape() as tape:
        last_conv_layer = new_model.get_layer('conv2')
        iterate = tf.keras.models.Model([new_model.inputs], [new_model.output, last_conv_layer.output])
        model_out, last_conv_layer = iterate(x_test)
        class_out = model_out[:, np.argmax(model_out[0])]
        grads = tape.gradient(class_out, last_conv_layer)
        pooled_grads = tf.keras.backend.mean(grads, axis=(0, 1, 2))
        
    heatmap = tf.reduce_mean(tf.multiply(pooled_grads, last_conv_layer), axis=-1)
    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap)
    df2 = pd.DataFrame(heatmap.transpose())
    if (len(heatmap_df)== 0): 
        heatmap_df= df2
    else:
        heatmap_df[k]=df2
    
    df2signal= pd.DataFrame(x_test_one.transpose()) 
    
    if (len(signal_df)== 0): 
        signal_df= df2signal
    else:
        signal_df[k]=df2signal

ValueError: ignored